# Auto-Atención o "*Self-Attention*"

## ¿A qué nos referimos con atención?

<center>
<img src="https://i.pinimg.com/originals/c7/63/dc/c763dc75719cd468551402142043f69b.gif" alt="source: https://www.pinterest.com/pin/464574517802151786/" width="35%">
<br>
<a href="https://www.pinterest.com/pin/464574517802151786/" target="_top">Button Ball Bounce.</a>
</center>

---


La **atención** es un proceso que involucra un **enfoque selectivo** de una o pocas cosas mientras se ignoran otras. Es un **mecanismo** inspirado en la forma en que creemos funciona la vista humana.

Por ejemplo, en la <u>animación anterior</u> es muy probable que una persona primero se enfoque en la bola que rebota, y luego note los botones que la bola presiona.

Otro ejemplo práctico sería cuando abrimos nuestra bandeja de correo electrónico, tendemos a fijarnos únicamente en el correo no leído, ignorando los que ya hemos procesado.

![picture](http://www.cs.virginia.edu/~pc9za/riiaa_2020/1.png)

Una premisa de este mecanismo, es que si una persona nota que el objeto al que necesita prestar atención usualmente se ubica en una parte específica, aprenderá que es muy probable que ese objeto siga apareciendo en esa parte, y por lo tanto va a tender a enfocar su vista en dicha área.

![picture](http://www.cs.virginia.edu/~pc9za/riiaa_2020/2.png)

## ¿Cómo se calcula?

<center>
<img src="http://www.cs.virginia.edu/~pc9za/riiaa_2020/3.PNG" alt="Figure 2: https://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf" width="50%">
<br>
<a href="https://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf" target="_top">Attention is all you need.</a>
<br>
</center>
<br>

Un módulo de *self-attention* toma *n* entradas y devuelve *n* salidas.

El mecanismo permite que las entradas interactúen entre sí y descubran a quién deben prestar más atención, así, relacionan diferentes posiciones de una secuencia para calcular una sola representación de esa secuencia [1].

<center>
<img src="https://d2l.ai/_images/attention.svg" alt="Figure 10.3.2: https://d2l.ai/chapter_attention-mechanisms/transformer.html" width="35%">
<br>
<a href="https://d2l.ai/chapter_attention-mechanisms/attention.html" target="_top">Attention Mechanisms.</a>
<br>
</center>
<br>

Podemos describir los parámetros de atención como una proyección de una consulta (**Q**uery) para una serie de pares clave-valor (**K**ey--**V**alue).

Se realiza principalmente en tres pasos:

*  Primero, tomamos la consulta (**Q**) y cada clave (**K**) y calculamos la similitud entre las dos para obtener los pesos.
*  Luego usamos una función softmax para normalizar estos pesos.
*  Finalmente ponderamos estos pesos junto con los valores correspondientes para obtener la atención final.

Daremos un recorrido por el algoritmo, paso por paso.

### Entradas y pesos

Definamos primero mi entrada, una matriz de 3x6, de manera que cada fila, represente un embedding word:
<center>
<img src="http://www.cs.virginia.edu/~pc9za/riiaa_2020/4.1.png" alt="input" width="25%">
<br>
</center>

<!-- Es decir, voy a tener 3 vectores cada uno con una dimension de 6:
<center>
<img src="http://www.cs.virginia.edu/~pc9za/riiaa_2020/4.2.png" alt="input" width="85%">
<br>
</center> -->

In [ ]:
import torch

x = [[2, 0, 1, 0, 3, 0], # e1
     [0, 1, 5, 1, 0, 1], # e2
     [1, 0, 1, 0, 1, 0]] # e3
x = torch.tensor(x, dtype=torch.float32)
print (x)

tensor([[1., 0., 1., 0., 3., 0.],
        [0., 2., 0., 2., 0., 1.],
        [1., 1., 1., 1., 1., 1.]])


Ahora vamos a definir un set de valores para mis pesos (**W**eights) correspondientes a mi consulta (**Q**uery), clave (**K**ey) y valor (**V**alue).

<center>
<img src="http://www.cs.virginia.edu/~pc9za/riiaa_2020/4.3.png" alt="pesos" width="60%">
<br>
</center>

*Estos pesos son aprendidos durante el entrenamiento de la red, aca definimos estos valores ficticios para ir paso a paso por el algoritmo*

In [ ]:
w_query = [[1, 1, 1],
           [0, 0, 0],
           [0, 0, 1],
           [0, 0, 1],
           [1, 0, 1],
           [0, 1, 1]]
w_key = [[1, 1, 0],
         [1, 0, 1],
         [0, 1, 0],
         [0, 0, 1],
         [0, 0, 1],
         [0, 1, 0]]
w_value = [[0, 0, 5],
           [0, 1, 0],
           [4, 1, 0],
           [0, 0, 5],
           [0, 4, 0],
           [2, 1, 0]]
w_query = torch.tensor(w_query, dtype=torch.float32)
w_key = torch.tensor(w_key, dtype=torch.float32)
w_value = torch.tensor(w_value, dtype=torch.float32)

print("W^Q:\n{}\n".format(w_query))
print("W^K:\n{}\n".format(w_key))
print("W^V:\n{}\n".format(w_value))

W^Q:
tensor([[1., 1., 1.],
        [0., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 1.],
        [0., 1., 1.]])

W^K:
tensor([[1., 1., 0.],
        [1., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.]])

W^V:
tensor([[0., 0., 5.],
        [0., 1., 0.],
        [4., 1., 0.],
        [0., 0., 5.],
        [0., 4., 0.],
        [2., 1., 0.]])



### Obtenemos Q, K y V

Como ya tenemos los pesos, ahora podemos computar los vectores correspondientes al **Q**uery, **K**ey y **V**alue para cada vector de entrada.

<center>
<img src="http://www.cs.virginia.edu/~pc9za/riiaa_2020/4.4.png" alt="pesos" width="50%">
<br>
</center>

In [ ]:
querys = torch.matmul(x, w_query)
keys = torch.matmul(x, w_key)
values = torch.matmul(x, w_value)

print("Queries:\n{}".format(querys))
print("Keys:\n{}".format(keys))
print("Values:\n{}".format(values))

Queries:
tensor([[4., 1., 5.],
        [0., 1., 3.],
        [2., 2., 5.]])
Keys:
tensor([[1., 2., 3.],
        [2., 1., 4.],
        [2., 3., 3.]])
Values:
tensor([[ 4., 13.,  5.],
        [ 2.,  3., 10.],
        [ 6.,  7., 10.]])


### Puntaje de Atencion

Ahora, para obtener las puntuaciones de atencion (**A**ttention **S**cores -**AS**-), vamos a calcular el producto escalar entre los **Q**ueries y los **K**eys

<center>
<img src="http://www.cs.virginia.edu/~pc9za/riiaa_2020/4.5.png" alt="pesos" width="35%">
<br>
</center>

In [ ]:
attn_scores = torch.matmul(querys, keys.t())
print("Puntuaciones de Atencion:\n{}".format(attn_scores))

Puntuaciones de Atencion:
tensor([[21., 29., 26.],
        [11., 13., 12.],
        [21., 26., 25.]])


### Salida
Finalmente calculamos el **softmax** de estos Attention Scores y multiplicamos por los **V**alues. 

<u>**El Softmax**</u> normaliza las puntuaciones, forzando que cada elemento en la fila esté dentro de un intarvalo que va de 0 a 1, y que su suma sea de 1 (distribucion de probabilidades). Esta puntuación va a ser muy alta en la posición de nuestra la palabra actual, pero además a veces es útil para interpretar a qué otra palabra el modelo presta atención, indicándonos qué es relevante para la palabra actual.

<center>
<img src="http://www.cs.virginia.edu/~pc9za/riiaa_2020/4.6.png" alt="pesos" width="35%">
<br>
</center>

In [ ]:
attn_scores_softmax = torch.softmax(attn_scores / querys.shape[-1], dim=-1)
print("Attention Scores despues del Softmax:\n{}\n".format(attn_scores_softmax))

# rendondiemos los valores para que sean mas faciles de leer...
attn_scores_softmax = torch.round(attn_scores_softmax * 10**1) / (10**1)

print("Attention Scores despues del Softmax (redondeados):\n{}".format(attn_scores_softmax))

Attention Scores despues del Softmax:
tensor([[0.0483, 0.6957, 0.2559],
        [0.2302, 0.4484, 0.3213],
        [0.0991, 0.5248, 0.3761]])

Attention Scores despues del Softmax (redondeados):
tensor([[0.0000, 0.7000, 0.3000],
        [0.2000, 0.4000, 0.3000],
        [0.1000, 0.5000, 0.4000]])


In [ ]:
output = torch.matmul(attn_scores_softmax, values)

print("Salida:\n{}".format(output))

Salida:
tensor([[ 3.2000,  4.2000, 10.0000],
        [ 3.4000,  5.9000,  8.0000],
        [ 3.8000,  5.6000,  9.5000]])


O de manera alternativa, podemos computar nuestra salida de la siguiente forma:

In [ ]:
weighted_values = attn_scores_softmax.T[:,:,None] * values[:,None]

print("Attention Scores multiplicados por los valores:\n{}".format(weighted_values))

Attention Scores multiplicados por los valores:
tensor([[[0.0000, 0.0000, 0.0000],
         [0.8000, 2.6000, 1.0000],
         [0.4000, 1.3000, 0.5000]],

        [[1.4000, 2.1000, 7.0000],
         [0.8000, 1.2000, 4.0000],
         [1.0000, 1.5000, 5.0000]],

        [[1.8000, 2.1000, 3.0000],
         [1.8000, 2.1000, 3.0000],
         [2.4000, 2.8000, 4.0000]]])


In [ ]:
outputs = weighted_values.sum(dim=0)

print("Salida:\n{}".format(outputs))

Salida:
tensor([[ 3.2000,  4.2000, 10.0000],
        [ 3.4000,  5.9000,  8.0000],
        [ 3.8000,  5.6000,  9.5000]])


De esta manera concluimos la primer seccion del taller, donde nos enfocamos en comprender cómo funcionan los mecanismos de atencion, y la matematica detras de estos.

## Referencias

[1] Vaswani, Ashish, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin. "Attention is all you need." In Advances in neural information processing systems, pp. 5998-6008. 2017.

